In [4]:
import MetaTrader5 as mt5 
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller 
from datetime import datetime
mt5.initialize()
# Replace following with your MT5 Account Login
account=51434456 # 
password="9UpBvVzc"
server = 'ICMarkets-Demo'

In [5]:
def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1[['time','open', 'high', 'low', 'close']].set_index('time')

def compute_spread(p1, p2, tf, x):
    data1 = get_rates(p1, tf, x)
    data2 = get_rates(p2, tf, x)
    merged = data1.join(data2, lsuffix="_x", rsuffix="_y")
    spread = merged['close_x'] - merged['close_y']
    return spread.dropna()

def adf_test(spread):
    '''Runs ADF test on a spread series'''
    result = adfuller(spread)
    return {'ADF Statistic': result[0], 'p-value': result[1], 'Critical Values': result[4]}

# Correlation Mean Reversion

If correlation diverges between two pairs of assets, price of the two assets will move oppositely. If correlation is mean reverting, you can profit from the correlation reverting back to its mean. Identify the leading and lagging pair for both high and low correlation. 
- Go long on the lagging
- Go short on the leader

In [106]:
def get_pair_correlations(symbol1, symbol2, window):
    s1 = str(symbol1)
    s2 = str(symbol2)
    symbol1 = get_rates(symbol1, mt5.TIMEFRAME_H4, 5000)
    symbol2 = get_rates(symbol2, mt5.TIMEFRAME_H4, 5000)

    combined_df = pd.concat([symbol1['close'].rename(f'{s1}_close'),
                             symbol2['close'].rename(f'{s2}_close')], axis=1)

    window_size = window  # Change this to the size of the window you want
    combined_df['rolling_corr'] = combined_df[f'{s1}_close'].rolling(window=window_size).corr(combined_df[f'{s2}_close'])
    # combined_df['rolling_corr'].iloc[0:200].plot()
    combined_df[f'{s1}_return'] = combined_df[f'{s1}_close'].pct_change()
    combined_df[f'{s2}_return'] = combined_df[f'{s2}_close'].pct_change()
    combined_df['diff'] = combined_df[f'{s1}_return'] - combined_df[f'{s2}_return']
    combined_df['rolling_corr_returns'] = combined_df['rolling_corr'].rolling(window=window_size).corr(combined_df['diff'])
    combined_df['shifted_rolling_corr_returns'] = combined_df['rolling_corr_returns'].shift(1)
    combined_df[f'{s1}_rolling_var'] = combined_df[f'{s1}_close'].rolling(window=3).var()
    combined_df[f'{s2}_rolling_var'] = combined_df[f'{s2}_close'].rolling(window=3).var()
    return combined_df.dropna()

In [115]:
df = get_pair_correlations('EURUSD.a', 'GBPUSD.a', 5)
df.corr()

,EURUSD.a_close,GBPUSD.a_close,rolling_corr,EURUSD.a_return,GBPUSD.a_return,diff,rolling_corr_returns,shifted_rolling_corr_returns,EURUSD.a_rolling_var,GBPUSD.a_rolling_var
EURUSD.a_close,1.000000,0.925048,-0.179678,0.014373,0.019167,-0.010635,0.001304,0.002740,-0.204263,-0.160184
GBPUSD.a_close,0.925048,1.000000,-0.162128,0.005010,0.020452,-0.023356,0.013478,0.014825,-0.205282,-0.204098
rolling_corr,-0.179678,-0.162128,1.000000,-0.009190,-0.000960,-0.009316,0.026594,0.026449,0.161886,0.100628
EURUSD.a_return,0.014373,0.005010,-0.009190,1.000000,0.719623,0.136002,-0.026151,0.049530,0.004719,-0.004052
GBPUSD.a_return,0.019167,0.020452,-0.000960,0.719623,1.000000,-0.590043,0.063716,0.038396,0.003644,-0.067109
diff,-0.010635,-0.023356,-0.009316,0.136002,-0.590043,1.000000,-0.121316,0.002808,0.000288,0.091039
rolling_corr_returns,0.001304,0.013478,0.026594,-0.026151,0.063716,-0.121316,1.000000,0.583276,0.000615,0.000871
shifted_rolling_corr_returns,0.002740,0.014825,0.026449,0.049530,0.038396,0.002808,0.583276,1.000000,0.002488,-0.004386
EURUSD.a_rolling_var,-0.204263,-0.205282,0.161886,0.004719,0.003644,0.000288,0.000615,0.002488,1.000000,0.506087
GBPUSD.a_rolling_var,-0.160184,-0.204098,0.100628,-0.004052,-0.067109,0.091039,0.000871,-0.004386,0.506087,1.000000


In [116]:
from statsmodels.tsa.stattools import grangercausalitytests

In [117]:
df.head()

,EURUSD.a_close,GBPUSD.a_close,rolling_corr,EURUSD.a_return,GBPUSD.a_return,diff,rolling_corr_returns,shifted_rolling_corr_returns,EURUSD.a_rolling_var,GBPUSD.a_rolling_var
time,,,,,,,,,,
2020-08-21 00:00:00,1.18743,1.32312,0.979027,0.001104,0.001294,-0.000190,-0.255634,-0.123829,1.041033e-06,0.000005
2020-08-21 04:00:00,1.18770,1.32417,0.983743,0.000227,0.000794,-0.000566,-0.140167,-0.255634,7.142333e-07,0.000002
2020-08-21 08:00:00,1.18127,1.31997,0.679592,-0.005414,-0.003172,-0.002242,0.899685,-0.140167,1.322723e-05,0.000005
2020-08-21 12:00:00,1.17850,1.31076,0.912234,-0.002345,-0.006977,0.004632,0.423561,0.899685,2.227630e-05,0.000047
2020-08-21 16:00:00,1.17724,1.30779,0.939277,-0.001069,-0.002266,0.001197,0.414018,0.423561,4.250233e-06,0.000040


In [125]:
adf_test(df['EURUSD.a_rolling_var'])

{'ADF Statistic': -8.373123544236513,
 'p-value': 2.629429165975397e-13,
 'Critical Values': {'1%': -3.4316688241926,
  '5%': -2.8621227766748327,
  '10%': -2.5670802129039485}}

In [124]:
for i in grangercausalitytests(df[['EURUSD.a_rolling_var', 'diff']], 25):
    print(f'Doing {i}')


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=3.0809  , p=0.0793  , df_denom=4987, df_num=1
ssr based chi2 test:   chi2=3.0828  , p=0.0791  , df=1
likelihood ratio test: chi2=3.0818  , p=0.0792  , df=1
parameter F test:         F=3.0809  , p=0.0793  , df_denom=4987, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=4.9892  , p=0.0068  , df_denom=4984, df_num=2
ssr based chi2 test:   chi2=9.9883  , p=0.0068  , df=2
likelihood ratio test: chi2=9.9783  , p=0.0068  , df=2
parameter F test:         F=4.9892  , p=0.0068  , df_denom=4984, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=3.3305  , p=0.0187  , df_denom=4981, df_num=3
ssr based chi2 test:   chi2=10.0055 , p=0.0185  , df=3
likelihood ratio test: chi2=9.9955  , p=0.0186  , df=3
parameter F test:         F=3.3305  , p=0.0187  , df_denom=4981, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=2.9137  , p=0.

### Based on the above findings:
Next steps:
1. Picking a direction; if 'diff' is positive, EURUSD may outperform GBPUSD, vice versa. 
2. You can forecast each symbol individually since there's a granger-casual relationship present.

Models to try:
- ARIMA
- LSTM 

In [24]:
def send_order(symbol, side, lot, comment):

    if side.lower() == 'sell':
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    elif side.lower() == 'buy':
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "price": price,
        "deviation": 5,
        "magic": 234000,
        "comment": comment,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    result

In [42]:
def close_position(position):

    tick = mt5.symbol_info_tick(position.symbol)

    request = {
        "action" : mt5.TRADE_ACTION_DEAL,
        "position": position.ticket,
        "symbol": position.symbol,
        "volume": position.volume,
        "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
        "price": tick.ask if position.type == 1 else tick.bid,
        "deviation": 20,
        "magic": 100,
        "comment": 'Regres Close',
        'type_time': mt5.ORDER_TIME_GTC,
        'type_filling':mt5.ORDER_FILLING_IOC,

        }
    result = mt5.order_send(request)
    
def check_close_position(trade_position):
    from datetime import datetime
    from datetime import datetime, timedelta
    # Your logic to close the position
    ticket = trade_position.ticket
    time = datetime.fromtimestamp(trade_position.time)
    print(time)
    symbol = trade_position.symbol
    comment = trade_position.comment
    # print(f"Checking {symbol} of {comment} to close")
    # Sample trade position (replace this with the actual object you get)
    position = {
        'ticket': ticket,
        'time': time,  # This would be the actual Unix timestamp
        'Symbol': symbol,
        'Comment': comment,
    }

    local_to_utc_offset = timedelta(hours= +3)

    # Get the current time in GMT+3
    current_time_gmt_plus_3 = datetime.now() 

    # Convert it to UTC
    current_time_utc = current_time_gmt_plus_3 + local_to_utc_offset

    # Convert to Unix timestamp
    current_unix_time = int(current_time_utc.timestamp())

    # Calculate the time difference in seconds
    time_difference = current_unix_time - int(i.time)
    
    # Check if 4 hours or more have passed (4 hours = 4 * 60 * 60 seconds)
    if time_difference >= 3600:
        print(f"Time difference is {(time_difference)} minutes. Closing {symbol} of {comment}")
        close_position(trade_position)
    else:
        print(f"Position {symbol} of {comment} has been open for less than 4 hours. Time open: {round(round((time_difference / 60),2) / 60), 4} hrs")

In [43]:
positions = mt5.positions_get()
for i in positions:
    print(f"Checking {i.symbol}")
    check_close_position(i)

Checking EURUSD.a
2023-10-30 09:34:12
Time difference is 12858 minutes. Closing EURUSD.a of Correl_Rev_D
Checking GBPUSD.a
2023-10-30 09:34:13
Time difference is 12858 minutes. Closing GBPUSD.a of Correl_Rev_D
Checking EURUSD.a
2023-10-30 09:34:14
Time difference is 12857 minutes. Closing EURUSD.a of Correl_Rev_EU
Checking GBPUSD.a
2023-10-30 09:34:15
Time difference is 12857 minutes. Closing GBPUSD.a of Correl_Rev_GU
Checking AUDUSD.a
2023-10-30 10:02:16
Time difference is 11176 minutes. Closing AUDUSD.a of Correl_Rev
Checking NZDUSD.a
2023-10-30 10:02:16
Time difference is 11177 minutes. Closing NZDUSD.a of Correl_Rev
Checking AUDUSD.a
2023-10-30 10:02:19
Time difference is 11174 minutes. Closing AUDUSD.a of Correl_Rev
Checking NZDUSD.a
2023-10-30 10:02:19
Time difference is 11175 minutes. Closing NZDUSD.a of Correl_Rev
Checking AUDUSD.a
2023-10-30 10:02:20
Time difference is 11174 minutes. Closing AUDUSD.a of Correl_Rev
Checking NZDUSD.a
2023-10-30 10:02:21
Time difference is 11174 

In [14]:
pair1 = 'AUDUSD.a'
pair1 + '_return'

'AUDUSD.a_return'

In [18]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

In [30]:
def arima_correl(pair1, pair2):
    df = get_pair_correlations(pair1, pair2, 5)
    
    lst = ['diff', f'{pair1}' + '_return', f'{pair2}' + '_return']
    for heading in lst:
        model_diff = ARIMA(df[heading], order=(2,0,0), exog=df['shifted_rolling_corr_returns'])
        model_diff_fit = model_diff.fit()
        yhat_diff = model_diff_fit.forecast(steps=1, exog=np.array([[df['shifted_rolling_corr_returns'].iloc[-2]]]))
        
        if heading == 'diff':
            lot = 1.75
        else:
            lot = 0.75
        
        if yhat_diff.values > 0.00003:
            send_order(pair1, 'sell', lot, 'Correl_Rev')
            send_order(pair2, 'buy', lot, 'Correl_Rev')
        elif yhat_diff.values < 0.00003:
            send_order(pair1, 'buy', lot, 'Correl_Rev')
            send_order(pair2, 'sell', lot, 'Correl_Rev')
        else:
            print("Inconclusive")

In [31]:
pairs = [['AUDUSD.a', 'NZDUSD.a'], ['EURUSD.a', 'GBPUSD.a']]

In [33]:
for i in pairs:
    print(i[0], i[1])

AUDUSD.a NZDUSD.a
EURUSD.a GBPUSD.a


In [28]:
arima_correl('AUDUSD.a', 'NZDUSD.a')

In [29]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

model_diff = ARIMA(df['diff'], order=(2,0,0), exog=df['shifted_rolling_corr_returns'])
model_diff_fit = model_diff.fit()
yhat_diff = model_diff_fit.forecast(steps=1, exog=np.array([[df['shifted_rolling_corr_returns'].iloc[-2]]]))
yhat_diff
# if yhat_diff.values > 0:
#     send_order('EURUSD.a', 'sell', 1.50, 'Correl_Rev')
#     send_order('GBPUSD.a', 'buy', 1.30, 'Correl_Rev')
# elif yhat_diff.values < 0:
#     send_order('EURUSD.a', 'buy', 1.50, 'Correl_Rev')
#     send_order('GBPUSD.a', 'sell', 1.30, 'Correl_Rev')

4991    0.000012
dtype: float64

In [120]:
model_diff = ARIMA(df['EURUSD.a_return'], order=(1,0,0), exog=df['shifted_rolling_corr_returns'])
model_diff_fit = model_diff.fit()
yhat_diff = model_diff_fit.forecast(steps=1, exog=np.array([[df['shifted_rolling_corr_returns'].iloc[-2]]]))

if yhat_diff.values > 0:
    send_order('EURUSD.a', 'buy', 1.50, 'Correl_Rev')
elif yhat_diff.values < 0:
    send_order('EURUSD.a', 'sell', 1.50, 'Correl_Rev')

In [121]:
model_diff = ARIMA(df['GBPUSD.a_return'], order=(1,0,0), exog=df['shifted_rolling_corr_returns'])
model_diff_fit = model_diff.fit()
yhat_diff = model_diff_fit.forecast(steps=1, exog=np.array([[df['shifted_rolling_corr_returns'].iloc[-2]]]))

if yhat_diff.values > 0:
    send_order('GBPUSD.a', 'buy', 1.30, 'Correl_Rev')
elif yhat_diff.values < 0:
    send_order('GBPUSD.a', 'sell', 1.30, 'Correl_Rev')

In [141]:
positions = mt5.positions_get()
for i in positions:
    if 'H1' in i.comment:
        print(f"Checking {i.symbol}")
        check_close_position(i)

Checking EURUSD.a
2023-10-26 15:10:14
Position EURUSD.a of H1Correl_Rev_D has been open for less than 4 hours. Time open: 0.013666666666666666 hrs
Checking GBPUSD.a
2023-10-26 15:10:15
Position GBPUSD.a of H1Correl_Rev_D has been open for less than 4 hours. Time open: 0.013333333333333334 hrs
Checking EURUSD.a
2023-10-26 15:10:16
Position EURUSD.a of H1Correl_Rev_EU has been open for less than 4 hours. Time open: 0.013000000000000001 hrs
Checking GBPUSD.a
2023-10-26 15:10:18
Position GBPUSD.a of H1Correl_Rev_GU has been open for less than 4 hours. Time open: 0.0125 hrs


In [102]:
hrly_lst = [['EU/GU'], ['AU/NU'], ['UJ/GJ']]
history = mt5.history_deals_get(datetime(2023,10,29), datetime.now())
history

(TradeDeal(ticket=373419117, order=512672344, time=1698650495, time_msc=1698650495760, type=1, entry=0, magic=234000, position_id=512672344, reason=3, volume=2.5, price=1.056, commission=-8.75, swap=0.0, profit=0.0, fee=0.0, symbol='EURUSD.a', comment='H1Correl_Rev_D', external_id=''),
 TradeDeal(ticket=373419123, order=512672359, time=1698650500, time_msc=1698650500653, type=0, entry=0, magic=234000, position_id=512672359, reason=3, volume=2.3, price=1.212, commission=-8.05, swap=0.0, profit=0.0, fee=0.0, symbol='GBPUSD.a', comment='H1Correl_Rev_D', external_id=''),
 TradeDeal(ticket=373419128, order=512672371, time=1698650504, time_msc=1698650504728, type=1, entry=0, magic=234000, position_id=512672371, reason=3, volume=2.5, price=1.056, commission=-8.75, swap=0.0, profit=0.0, fee=0.0, symbol='EURUSD.a', comment='H1Correl_Rev_EU', external_id=''),
 TradeDeal(ticket=373419134, order=512672378, time=1698650506, time_msc=1698650506891, type=1, entry=0, magic=234000, position_id=51267237

In [97]:
lst = []
for i in history:
    # print(i.comment)
    if i.symbol == 'EURUSD.a':
        if 'CO' in i.comment:
            lst.append(i)

In [98]:
lst

[TradeDeal(ticket=373629707, order=512929464, time=1698671667, time_msc=1698671667787, type=0, entry=0, magic=234000, position_id=512929464, reason=3, volume=1.25, price=1.05835, commission=-4.38, swap=0.0, profit=0.0, fee=0.0, symbol='EURUSD.a', comment='H1/CORARIMA df', external_id=''),
 TradeDeal(ticket=373629735, order=512929501, time=1698671670, time_msc=1698671670523, type=0, entry=0, magic=234000, position_id=512929501, reason=3, volume=0.75, price=1.05834, commission=-2.63, swap=0.0, profit=0.0, fee=0.0, symbol='EURUSD.a', comment='H1/CORARIMA EU', external_id=''),
 TradeDeal(ticket=373629749, order=512929517, time=1698671672, time_msc=1698671672380, type=0, entry=0, magic=234000, position_id=512929517, reason=3, volume=0.75, price=1.05832, commission=-2.63, swap=0.0, profit=0.0, fee=0.0, symbol='EURUSD.a', comment='H1/CORARIMA GU', external_id=''),
 TradeDeal(ticket=373747730, order=513072425, time=1698681610, time_msc=1698681610444, type=0, entry=0, magic=234000, position_id=

In [84]:
for i in hrly_lst:
    print(i[0])

EU/GU
AU/NU
UJ/GJ


In [79]:
hrly_lst = [['EU/GU'], ['AU/NU'], ['UJ/GJ']]
history = mt5.history_deals_get(datetime(2023,10,29), datetime.now())

for i in hrly_lst:
    for position in history:
        if ('H1' in position.comment or 'Correl' in position.comment or 'Corr' in position.comment) and (position.symbol == 'EURUSD.a' or position.symbol == 'GBPUSD.a'):
            for inner_list in hrly_lst:
                if inner_list[0] == 'EU/GU':
                    lst = []
                    lst.append(position.profit)
                    lst.append(position.symbol)
                    lst.append(position.comment)
                    # print(lst)
                    inner_list.append(lst)

In [80]:
hrly_lst

[['EU/GU',
  [0.0, 'EURUSD.a', 'H1Correl_Rev_D'],
  [0.0, 'GBPUSD.a', 'H1Correl_Rev_D'],
  [0.0, 'EURUSD.a', 'H1Correl_Rev_EU'],
  [0.0, 'GBPUSD.a', 'H1Correl_Rev_GU'],
  [0.0, 'GBPUSD.a', 'H1Correl_Rev_D'],
  [0.0, 'EURUSD.a', 'H1Correl_Rev_EU'],
  [0.0, 'EURUSD.a', 'Correl_Rev_D'],
  [0.0, 'GBPUSD.a', 'Correl_Rev_D'],
  [0.0, 'EURUSD.a', 'H1Correl_Rev_D'],
  [0.0, 'GBPUSD.a', 'H1Correl_Rev_GU'],
  [0.0, 'EURUSD.a', 'Correl_Rev_D'],
  [0.0, 'GBPUSD.a', 'Correl_Rev_D'],
  [0.0, 'EURUSD.a', 'Correl_Rev_EU'],
  [0.0, 'GBPUSD.a', 'Correl_Rev_GU'],
  [0.0, 'EURUSD.a', 'H1Correl_Rev_D'],
  [0.0, 'GBPUSD.a', 'H1Correl_Rev_D'],
  [0.0, 'EURUSD.a', 'H1Correl_Rev_EU'],
  [0.0, 'GBPUSD.a', 'H1Correl_Rev_GU'],
  [0.0, 'EURUSD.a', 'Correl_Rev'],
  [0.0, 'EURUSD.a', 'Correl_Rev'],
  [0.0, 'GBPUSD.a', 'Correl_Rev'],
  [0.0, 'EURUSD.a', 'Correl_Rev'],
  [0.0, 'GBPUSD.a', 'Correl_Rev'],
  [0.0, 'EURUSD.a', 'Correl_Rev'],
  [0.0, 'GBPUSD.a', 'Correl_Rev'],
  [0.0, 'EURUSD.a', 'Correl_Rev'],
  [0.0, 'GB